In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Llama 3.1 Finetuning

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_pytorch_llama3_1_finetuning.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_llama3_1_finetuning.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates finetuning and deploying Llama 3.1 models with Vertex AI. All of the examples in this notebook use parameter efficient finetuning methods [PEFT (LoRA)](https://github.com/huggingface/peft) to reduce training and storage costs. LoRA (Low-Rank Adaptation) is one approach of Parameter Efficient FineTuning (PEFT), where pretrained model weights are frozen and rank decomposition matrices representing the change in model weights are trained during finetuning. Read more about LoRA in the following publication: [Hu, E.J., Shen, Y., Wallis, P., Allen-Zhu, Z., Li, Y., Wang, S., Wang, L. and Chen, W., 2021. Lora: Low-rank adaptation of large language models. *arXiv preprint arXiv:2106.09685*](https://arxiv.org/abs/2106.09685).

After finetuning, we can deploy models on Vertex with GPU.


### Objective

- Finetune Llama 3.1 models with Vertex AI Custom Training Jobs.
- Evaluate the finetuned model using [lm-evaluation-harness](https://github.com/EleutherAI/lm-evaluation-harness).
- Deploy finetuned Llama 3.1 models on Vertex AI Prediction.
- Send prediction requests to your finetuned Llama 3.1 models.

### File a bug

File a bug on [GitHub](https://github.com/GoogleCloudPlatform/vertex-ai-samples/issues/new) if you encounter any issue with the notebook.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

In [ ]:
# @title Install Python Packages for Finetuning

# @markdown 1. Install packages to validate dataset with template.
! pip install --upgrade --quiet gcsfs==2024.3.1
! pip install --upgrade --quiet accelerate==0.34.2
! pip install --upgrade --quiet transformers==4.47.1
! pip install --upgrade --quiet datasets==2.20.0
! pip install --upgrade --quiet google-cloud-aiplatform==1.130.0

In [ ]:
# @title Setup Google Cloud project

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. For finetuning, **[click here](https://console.cloud.google.com/iam-admin/quotas?location=us-central1&metric=aiplatform.googleapis.com%2Frestricted_image_training_nvidia_a100_80gb_gpus)** to check if your project already has the required 8 Nvidia A100 80 GB GPUs in the us-central1 region. If yes, then run this notebook in the us-central1 region. If you do not have 8 Nvidia A100 80 GPUs or have more GPU requirements than this, then schedule your job with Nvidia H100 GPUs via Dynamic Workload Scheduler using [these instructions](https://cloud.google.com/vertex-ai/docs/training/schedule-jobs-dws). For Dynamic Workload Scheduler, check the [us-central1](https://console.cloud.google.com/iam-admin/quotas?location=us-central1&metric=aiplatform.googleapis.com%2Fcustom_model_training_preemptible_nvidia_h100_gpus) or [europe-west4](https://console.cloud.google.com/iam-admin/quotas?location=europe-west4&metric=aiplatform.googleapis.com%2Fcustom_model_training_preemptible_nvidia_h100_gpus) quota for Nvidia H100 GPUs. If you do not have enough GPUs, then you can follow [these instructions](https://cloud.google.com/docs/quotas/view-manage#viewing_your_quota_console) to request quota.

# @markdown 3. For serving, **[click here](https://console.cloud.google.com/iam-admin/quotas?location=us-central1&metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_l4_gpus)** to check if your project already has the required 1 L4 GPU in the us-central1 region.  If yes, then run this notebook in the us-central1 region. If you need more L4 GPUs for your project, then you can follow [these instructions](https://cloud.google.com/docs/quotas/view-manage#viewing_your_quota_console) to request more. Alternatively, if you want to run predictions with A100 80GB or H100 GPUs, we recommend using the regions listed below. **NOTE:** Make sure you have associated quota in selected regions. Click the links to see your current quota for each GPU type: [Nvidia A100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_a100_80gb_gpus), [Nvidia H100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_h100_gpus).

# @markdown > | Machine Type | Accelerator Type | Recommended Regions |
# @markdown | ----------- | ----------- | ----------- |
# @markdown | a2-ultragpu-1g | 1 NVIDIA_A100_80GB | us-central1, us-east4, europe-west4, asia-southeast1, us-east4 |
# @markdown | a3-highgpu-2g | 2 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-4g | 4 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-8g | 8 NVIDIA_H100_80GB | us-central1, europe-west4, us-west1, asia-southeast1 |

# @markdown 4. **[Optional]** [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1"). If not set, a unique GCS bucket will be created instead.

BUCKET_URI = "gs://"  # @param {type:"string"}

# @markdown 5. **[Optional]** Set region. If not set, the region will be set automatically according to Colab Enterprise environment.

REGION = ""  # @param {type:"string"}

# Import the necessary packages.
! rm -rf vertex-ai-samples && git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git
! cd vertex-ai-samples && git reset --hard 7ae13b346a72ee2a2dc8152dd40c6ddd72d6c810

import datetime
import importlib
import os
import uuid
from typing import Tuple

from google.cloud import aiplatform
from google.cloud.aiplatform.compat.types import \
    custom_job as gca_custom_job_compat

common_util = importlib.import_module(
    "vertex-ai-samples.notebooks.community.model_garden.docker_source_codes.notebook_util.common_util"
)

models, endpoints = {}, {}

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    if not os.environ.get("GOOGLE_CLOUD_REGION"):
        raise ValueError(
            "REGION must be set. See"
            " https://cloud.google.com/vertex-ai/docs/general/locations for"
            " available cloud locations."
        )
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, change the value yourself below.
now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])

if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}-{str(uuid.uuid4())[:4]}"
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )
print(f"Using this GCS Bucket: {BUCKET_URI}")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
MODEL_BUCKET = os.path.join(BUCKET_URI, "llama3_1")


# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Gets the default SERVICE_ACCOUNT.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this default Service Account:", SERVICE_ACCOUNT)


# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

! gcloud config set project $PROJECT_ID
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/storage.admin"
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/aiplatform.user"

In [ ]:
# @title Access Llama 3.1 models

# @markdown For GPU based finetuning and serving, choose between accessing Llama 3.1 models on [Hugging Face](https://huggingface.co/)
# @markdown or Vertex AI as described below.

# @markdown If you already obtained access to Llama 3.1 models on [Hugging Face](https://huggingface.co/), you can load models from there.
# @markdown Alternatively, you can also load the original Llama 3.1 models for finetuning and serving from Vertex AI after accepting the agreement.

# @markdown **Only select and fill one of the following sections.**
# It is recommended to use "Google Cloud" for 405B model since it can be downloaded faster.
LOAD_MODEL_FROM = "Google Cloud"  # @param ["Hugging Face", "Google Cloud"] {isTemplate:true}

# @markdown ---

# @markdown ### Access Llama 3.1 models on Hugging Face for GPU based finetuning and serving
# @markdown You must provide a Hugging Face User Access Token (with read access) to access the Llama 3.1 models. You can follow the [Hugging Face documentation](https://huggingface.co/docs/hub/en/security-tokens) to create a **read** access token and put it in the `HF_TOKEN` field below.

HF_TOKEN = ""  # @param {type:"string", isTemplate:true}
# @markdown *--- Or ---*
# @markdown ### Access Llama 3.1 models on Vertex AI for GPU based serving
# @markdown The original models from Meta are converted into the Hugging Face format for serving in Vertex AI.
# @markdown Accept the model agreement to access the models:
# @markdown 1. Open the [Llama 3.1 model card](https://console.cloud.google.com/vertex-ai/publishers/meta/model-garden/llama3_1) from [Vertex AI Model Garden](https://cloud.google.com/model-garden).
# @markdown 2. Review and accept the agreement in the pop-up window on the model card page. If you have previously accepted the model agreement, there will not be a pop-up window on the model card page and this step is not needed.
# @markdown 3. After accepting the agreement of Llama 3.1, a `gs://` URI containing Llama 3.1 pretrained and finetuned models will be shared.
# @markdown 4. Paste the URI in the `VERTEX_AI_MODEL_GARDEN_LLAMA3_1` field below.

VERTEX_AI_MODEL_GARDEN_LLAMA3_1 = ""  # @param {type:"string", isTemplate:true}
if LOAD_MODEL_FROM == "Hugging Face":
    assert (
        HF_TOKEN
    ), "Provide a read HF_TOKEN to load models from Hugging Face, or select a different model source."
else:
    assert (
        VERTEX_AI_MODEL_GARDEN_LLAMA3_1
    ), "Click the agreement of Llama3.1 in Vertex AI Model Garden, and get the GCS path of the model artifacts."

MODEL_BUCKET = VERTEX_AI_MODEL_GARDEN_LLAMA3_1

## Finetune with HuggingFace PEFT and deploy with vLLM on GPUs

In [ ]:
# @title Set dataset

# @markdown Use the Vertex AI SDK to create and run the custom training jobs.

# @markdown This notebook uses [timdettmers/openassistant-guanaco](https://huggingface.co/datasets/timdettmers/openassistant-guanaco) dataset as an example.
# @markdown You can set `train_dataset` to any existing [Hugging Face dataset](https://huggingface.co/datasets) name, and set `train_column` to the name of the dataset column containing training data. The [timdettmers/openassistant-guanaco](https://huggingface.co/datasets/timdettmers/openassistant-guanaco) has only one column `text`, and therefore we set `train_column` to `text` in this notebook.

# @markdown ### (Optional) Prepare a custom JSONL dataset for finetuning

# @markdown You can prepare a JSONL file where each line is a valid JSON string as your custom training dataset. For example, here is one line from the [timdettmers/openassistant-guanaco](https://huggingface.co/datasets/timdettmers/openassistant-guanaco) dataset:
# @markdown ```
# @markdown {"text": "### Human: Hola### Assistant: \u00a1Hola! \u00bfEn qu\u00e9 puedo ayudarte hoy?"}
# @markdown ```

# @markdown The JSON object has a key `text`, which should match `train_column`; The value should be one training data point, i.e. a string. After you prepared your JSONL file, you can either upload it to [Hugging Face datasets](https://huggingface.co/datasets) or [Google Cloud Storage](https://cloud.google.com/storage).

# @markdown - To upload a JSONL dataset to [Hugging Face datasets](https://huggingface.co/datasets), follow the instructions on [Uploading Datasets](https://huggingface.co/docs/hub/en/datasets-adding). Then, set `dataset_name` to the name of your newly created dataset on Hugging Face.

# @markdown - To upload a JSONL dataset to [Google Cloud Storage](https://cloud.google.com/storage), follow the instructions on [Upload objects from a filesystem](https://cloud.google.com/storage/docs/uploading-objects). Then, set `dataset_name` to the `gs://` URI to your JSONL file. For example: `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl`.

# @markdown Optionally update the `train_column` field below if your JSON objects use a key other than the default `text`.

# @markdown ### (Optional) Format your data with custom JSON template

# @markdown Sometimes, your dataset might have multiple text columns and you want to construct the training data with a template. You can prepare a JSON template in the following format:

# @markdown ```
# @markdown {
# @markdown   "description": "Template used by Llama 3.1, accepting text-bison format.",
# @markdown   "source": "https://cloud.google.com/vertex-ai/generative-ai/docs/models/tune-text-models-supervised#dataset-format",
# @markdown   "prompt_input": "<|start_header_id|>user<|end_header_id|>\n\n{input_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{output_text}<|eot_id|>",
# @markdown   "instruction_separator": "<|start_header_id|>user<|end_header_id|>\n\n",
# @markdown   "response_separator": "<|start_header_id|>assistant<|end_header_id|>\n\n"
# @markdown }
# @markdown ```

# @markdown As an example, the template above can be used to format the following training data (this line comes from `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl`):

# @markdown ```
# @markdown {"input_text":"TRANSCRIPT: \nREASON FOR EVALUATION:,\n\n LABEL:","output_text":"Chiropractic"}
# @markdown ```

# @markdown This example template simply concatenates `input_text` with `output_text` with some special tokens in between.
# @markdown
# @markdown To try such custom dataset, you can make the following changes:
# @markdown 1. Set `template` to `llama3-text-bison`
# @markdown 1. Set `train_dataset` to `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl`
# @markdown 1. Set `train_split` to `train`
# @markdown 1. Set `eval_dataset` to `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_eval_sample.jsonl`
# @markdown 1. Set `eval_split` to `train` (**NOT** `test`)
# @markdown 1. Set `train_column` as `input_text`.

# Template name or gs:// URI to a custom template.
template = "openassistant-guanaco"  # @param {type:"string"}

# Hugging Face dataset name or gs:// URI to a custom JSONL dataset.
train_dataset = "timdettmers/openassistant-guanaco"  # @param {type:"string"}
train_split = "train"  # @param {type:"string"}
eval_dataset = "timdettmers/openassistant-guanaco"  # @param {type:"string"}
eval_split = "test"  # @param {type:"string"}

# Name of the dataset column containing training text input.
train_column = "text"  # @param {type:"string"}
# Maximum sequence length.
max_seq_length = 4096  # @param{type:"integer"}

In [ ]:
# @title Set model

# @markdown Select a model variant of Llama 3.1.
base_model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"  # @param ["meta-llama/Meta-Llama-3.1-8B", "meta-llama/Meta-Llama-3.1-8B-Instruct", "meta-llama/Meta-Llama-3.1-70B", "meta-llama/Meta-Llama-3.1-70B-Instruct", "meta-llama/Meta-Llama-3.1-405B", "meta-llama/Meta-Llama-3.1-405B-Instruct", "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"] {isTemplate:true}
if LOAD_MODEL_FROM == "Google Cloud":
    pretrained_model_id = os.path.join(MODEL_BUCKET, base_model_id.split("/")[-1])
else:
    pretrained_model_id = base_model_id

In [ ]:
# @title Validate Dataset with Template

# @markdown This section validates the train and eval datasets with the template before starting the fine tuning process.

dataset_validation_util = importlib.import_module(
    "vertex-ai-samples.community-content.vertex_model_garden.model_oss.notebook_util.dataset_validation_util"
)

if dataset_validation_util.is_gcs_path(pretrained_model_id):
    # Download tokenizer.
    ! mkdir tokenizer
    ! gsutil cp {pretrained_model_id}/tokenizer.json ./tokenizer
    ! gsutil cp {pretrained_model_id}/config.json ./tokenizer
    tokenizer_path = "./tokenizer"
    access_token = ""
else:
    tokenizer_path = pretrained_model_id
    access_token = HF_TOKEN

tokenizer = dataset_validation_util.load_tokenizer(tokenizer_path, None, access_token)

# Validate the train dataset.
dataset_validation_util.validate_dataset_with_template(
    dataset_name=train_dataset,
    split=train_split,
    input_column=train_column,
    template=template,
    max_seq_length=max_seq_length,
    use_multiprocessing=False,
    tokenizer=tokenizer,
)

# Validate the eval dataset if it exists.
if eval_dataset:
    dataset_validation_util.validate_dataset_with_template(
        dataset_name=eval_dataset,
        split=eval_split,
        input_column=train_column,
        template=template,
        max_seq_length=max_seq_length,
        use_multiprocessing=False,
        tokenizer=tokenizer,
    )

In [ ]:
# @title Finetune

# @markdown This section demonstrates how to finetune the Llama 3.1 model on Vertex AI. It uses the Vertex AI SDK to create and run the custom training jobs.

# @markdown The training job takes approximately between 10 to 20 mins to set-up. Once done, the training job is expected to take around 75 mins with the default configurations. To find the training time, throughput, and memory usage of your training job, you can go to the training logs and check the log line of the last training epoch.

# @markdown **Note**:
# @markdown 1. We recommend setting `finetuning_precision_mode` to `4bit` because it enables using fewer hardware resources for finetuning.
# @markdown 1. If `max_steps > 0`, it takes precedence over `epochs`. One can set a small `max_steps` value to quickly check the pipeline.

# @markdown Acceletor type to use for training.
training_accelerator_type = "NVIDIA_A100_80GB"  # @param ["NVIDIA_A100_80GB", "NVIDIA_H100_80GB"]

# @markdown Set the Training Region. If not set, it will be set to default region.
TRAINING_REGION = ""  # @param {type: "string"}
if not TRAINING_REGION:
    TRAINING_REGION = REGION

aiplatform.init(location=TRAINING_REGION)

# The pre-built training docker image.
if training_accelerator_type == "NVIDIA_A100_80GB":
    repo = "us-docker.pkg.dev/vertex-ai-restricted"
    is_restricted_image = True
    is_dynamic_workload_scheduler = False
    dws_kwargs = {}
    if "405b" in base_model_id.lower():
        raise ValueError(
            "405B model is not supported with Nvidia A100 GPUs. Use Nvidia H100 GPUs instead."
        )
else:
    repo = "us-docker.pkg.dev/vertex-ai"
    is_restricted_image = False
    is_dynamic_workload_scheduler = True
    dws_kwargs = {
        "max_wait_duration": 5400,  # 90 minutes
        "scheduling_strategy": gca_custom_job_compat.Scheduling.Strategy.FLEX_START,
    }

TRAIN_DOCKER_URI = (
    f"{repo}/vertex-vision-model-garden-dockers/pytorch-peft-train:stable_20250705"
)

# Worker pool spec.
boot_disk_size_gb = 500
if training_accelerator_type == "NVIDIA_A100_80GB":
    per_node_accelerator_count = 8
    training_machine_type = "a2-ultragpu-8g"
elif training_accelerator_type == "NVIDIA_H100_80GB":
    per_node_accelerator_count = 8
    training_machine_type = "a3-highgpu-8g"
    if "405b" in base_model_id.lower():
        boot_disk_size_gb = 2000
else:
    raise ValueError(
        f"Recommended machine settings not found for: {training_accelerator_type}. To use another accelerator type, edit this code block to pass in an appropriate `training_machine_type`, `training_accelerator_type`, and `per_node_accelerator_count` by clicking `Show Code` and then modifying the code."
    )

# @markdown The number of nodes to use for this worker pool in distributed training.
replica_count = 1  # @param{type:"integer"}

# Set config file.
if replica_count == 1:
    config_file = "vertex_vision_model_garden_peft/llama_fsdp_8gpu.yaml"
elif replica_count <= 4:
    config_file = (
        "vertex_vision_model_garden_peft/"
        f"llama_hsdp_{replica_count * per_node_accelerator_count}gpu.yaml"
    )
else:
    raise ValueError(
        f"Recommended config settings not found for replica_count: {replica_count}."
    )

# @markdown Batch size for finetuning.
per_device_train_batch_size = 1  # @param{type:"integer"}
# @markdown Number of updates steps to accumulate the gradients for, before performing a backward/update pass.
gradient_accumulation_steps = 4  # @param{type:"integer"}
# @markdown Setting a positive `max_steps` here will override `num_train_epochs`.
max_steps = -1  # @param{type:"integer"}
num_train_epochs = 1.0  # @param{type:"number"}
# @markdown Precision mode for finetuning.
finetuning_precision_mode = "4bit"  # @param ["4bit", "8bit", "float16"]
# @markdown Learning rate.
learning_rate = 5e-5  # @param{type:"number"}
# @markdown The scheduler type to use.
lr_scheduler_type = "cosine"  # @param{type:"string"}
# @markdown LoRA parameters.
lora_rank = 16  # @param{type:"integer"}
lora_alpha = 32  # @param{type:"integer"}
lora_dropout = 0.05  # @param{type:"number"}
# Activates gradient checkpointing for the current model (may be referred to as activation checkpointing or checkpoint activations in other frameworks).
gradient_checkpointing = True
# Attention implementation to use in the model.
attn_implementation = "flash_attention_2"
# The optimizer for which to schedule the learning rate.
optimizer = "adamw_torch"
# Define the proportion of training to be dedicated to a linear warmup where learning rate gradually increases.
warmup_ratio = "0.01"
# The list or string of integrations to report the results and logs to.
report_to = "tensorboard"
# Number of updates steps before two checkpoint saves.
save_steps = 10
# Number of update steps between two logs.
logging_steps = save_steps

# @markdown Evaluation metrics to compute. Supported eval metrics: loss, perplexity, bleu, google_bleu, rouge1, rouge2, rougeL, rougeLsum.
eval_metric_name = "loss,perplexity,bleu"  # @param{type:"string"}
# @markdown Metric to use for best model selection. This will save the best checkpoint based on the eval metric.
metric_for_best_model = "perplexity"  # @param{type:"string"}

common_util.check_quota(
    project_id=PROJECT_ID,
    region=TRAINING_REGION,
    accelerator_type=training_accelerator_type,
    accelerator_count=per_node_accelerator_count * replica_count,
    is_for_training=True,
    is_restricted_image=is_restricted_image,
    is_dynamic_workload_scheduler=is_dynamic_workload_scheduler,
)

job_name = common_util.get_job_name_with_datetime("llama3_1-lora-train")

base_output_dir = os.path.join(STAGING_BUCKET, job_name)
# Create a GCS folder to store the LORA adapter.
lora_output_dir = os.path.join(base_output_dir, "adapter")
# Create a GCS folder to store the finetuned LORA adapter.
final_checkpoint = os.path.join(lora_output_dir, "node-0", "checkpoint-final")

# Add labels for the finetuning job.
labels = {
    "mg-source": "notebook",
    "mg-notebook-name": "model_garden_pytorch_llama3_1_finetuning.ipynb".split(".")[0],
}

labels["mg-tune"] = "publishers-meta-models-llama3-1"
versioned_model_id = base_model_id.split("/")[1].lower().replace(".", "-")
labels["versioned-mg-tune"] = f"{labels['mg-tune']}-{versioned_model_id}"

eval_args = [
    f"--eval_dataset={eval_dataset}",
    f"--eval_column={train_column}",
    f"--eval_template={template}",
    f"--eval_split={eval_split}",
    f"--eval_steps={save_steps}",
    f"--eval_metric_name={eval_metric_name}",
    f"--metric_for_best_model={metric_for_best_model}",
]

train_job_args = [
    f"--config_file={config_file}",
    "--task=instruct-lora",
    "--input_masking=True",
    f"--pretrained_model_name_or_path={pretrained_model_id}",
    f"--train_dataset={train_dataset}",
    f"--train_split={train_split}",
    f"--train_column={train_column}",
    f"--output_dir={lora_output_dir}",
    f"--per_device_train_batch_size={per_device_train_batch_size}",
    f"--gradient_accumulation_steps={gradient_accumulation_steps}",
    f"--lora_rank={lora_rank}",
    f"--lora_alpha={lora_alpha}",
    f"--lora_dropout={lora_dropout}",
    f"--max_steps={max_steps}",
    f"--max_seq_length={max_seq_length}",
    f"--learning_rate={learning_rate}",
    f"--lr_scheduler_type={lr_scheduler_type}",
    f"--precision_mode={finetuning_precision_mode}",
    f"--gradient_checkpointing={gradient_checkpointing}",
    f"--num_train_epochs={num_train_epochs}",
    f"--attn_implementation={attn_implementation}",
    f"--optimizer={optimizer}",
    f"--warmup_ratio={warmup_ratio}",
    f"--report_to={report_to}",
    f"--logging_output_dir={base_output_dir}",
    f"--save_steps={save_steps}",
    f"--logging_steps={logging_steps}",
    f"--train_template={template}",
    f"--huggingface_access_token={HF_TOKEN}",
] + eval_args

# Pass training arguments and launch job.
train_job = aiplatform.CustomContainerTrainingJob(
    display_name=job_name,
    container_uri=TRAIN_DOCKER_URI,
    labels=labels,
)

print("Running training job with args:")
print(" \\\n".join(train_job_args))
train_job.run(
    args=train_job_args,
    replica_count=replica_count,
    machine_type=training_machine_type,
    accelerator_type=training_accelerator_type,
    accelerator_count=per_node_accelerator_count,
    boot_disk_size_gb=boot_disk_size_gb,
    service_account=SERVICE_ACCOUNT,
    base_output_dir=base_output_dir,
    sync=False,  # Non-blocking call to run.
    **dws_kwargs,
)

# Wait until resource has been created.
train_job.wait_for_resource_creation()

print("LoRA adapter will be saved in:", lora_output_dir)
print("Final checkpoint will be saved in:", final_checkpoint)

# @markdown Click "Show Code" to see more details.

In [ ]:
# @title Run TensorBoard
# @markdown This section shows how to launch TensorBoard in a [Cloud Shell](https://cloud.google.com/shell/docs).
# @markdown 1. Click the Cloud Shell icon(![terminal](https://github.com/google/material-design-icons/blob/master/png/action/terminal/materialicons/24dp/1x/baseline_terminal_black_24dp.png?raw=true)) on the top right to open the Cloud Shell.
# @markdown 2. Copy the `tensorboard` command shown below by running this cell.
# @markdown 3. Paste and run the command in the Cloud Shell to launch TensorBoard.
# @markdown 4. Once the command runs (You may have to click `Authorize` if prompted), click the link starting with `http://localhost`.

# @markdown Note: You may need to wait around 10 minutes after the job starts in order for the TensorBoard logs to be written to the GCS bucket.
print(f"Command to copy: tensorboard --logdir {base_output_dir}/logs")

In [ ]:
# @title Select Evaluation Checkpoint

if train_job.end_time is None:
    print("Waiting for the training job to finish...")
    train_job.wait()
    print("The training job has finished.")

# @markdown The following checkpoints are available for evaluation:
! gcloud storage ls "{lora_output_dir}/node-0" | grep "checkpoint-"

In [ ]:
# @title Run Evaluation Job
# @markdown This section runs the evaluation using [lm-evaluation-harness](https://github.com/EleutherAI/lm-evaluation-harness) on the finetuned model. The evaluation takes approximately 20 mins to finish.

# The pre-built evaluation docker image for LM Evaluation Harness.
LM_EVAL_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-lm-evaluation-harness:20250410_1035_RC00"

# @markdown Set `RUN_EVALUATION` to False to skip the evaluation job.
RUN_EVALUATION = True  # @param {type:"boolean"}

# @markdown Set the Evaluation Region. If not set, it will be set to default region.
EVAL_REGION = ""  # @param {type: "string"}
if not EVAL_REGION:
    EVAL_REGION = REGION

aiplatform.init(location=EVAL_REGION)

if "8b" in base_model_id.lower():
    eval_machine_type = "g2-standard-24"
    eval_accelerator_type = "NVIDIA_L4"
    eval_accelerator_count = 2
    dws_kwargs = {
        "max_wait_duration": 10800,  # 180 minutes
        "scheduling_strategy": gca_custom_job_compat.Scheduling.Strategy.FLEX_START,
    }
    is_dynamic_workload_scheduler = True
elif "70b" in base_model_id.lower():
    eval_machine_type = "a2-ultragpu-4g"
    eval_accelerator_type = "NVIDIA_A100_80GB"
    eval_accelerator_count = 4
    dws_kwargs = {
        "max_wait_duration": 5400,  # 90 minutes
        "scheduling_strategy": gca_custom_job_compat.Scheduling.Strategy.FLEX_START,
    }
    is_dynamic_workload_scheduler = True
elif "405b" in base_model_id.lower():
    print(
        "405B model is not supported for evaluation. We will skip the evaluation job."
    )
    RUN_EVALUATION = False
else:
    raise ValueError(f"Unsupported model ID or GCS path: {base_model_id}.")

# @markdown Set `evaluation_checkpoint_dir` to an intermediate checkpoint from the above training job. If not set, the evaluation job will use the final checkpoint.
evaluation_checkpoint_dir = ""  # @param {type:"string"}
if not evaluation_checkpoint_dir:
    evaluation_checkpoint_dir = final_checkpoint

# @markdown Evaluation tasks to run.
eval_tasks = "coqa"  # @param {type:"string"}
# @markdown Model to use for evaluation.
model = "vllm"  # @param {type:"string"}
# @markdown Batch size for evaluation.
batch_size = "auto"  # @param {type:"string"}
apply_chat_template = True if "-Instruct" in pretrained_model_id else False
gpu_memory_utilization = 0.8
max_model_len = 8192  # Maximum context length.
eval_output_dir = os.path.join(base_output_dir, "lm_eval")

if RUN_EVALUATION:
    model_args = f"tensor_parallel_size={eval_accelerator_count},max_model_len={max_model_len},gpu_memory_utilization={gpu_memory_utilization},enforce_eager=True"
    lm_eval_job_args = [
        "--task=lm_eval",
        f"--model={model}",
        f"--eval_tasks={eval_tasks}",
        f"--pretrained_model_name_or_path={pretrained_model_id}",
        f"--model_args={model_args}",
        f'--lora_path={evaluation_checkpoint_dir.rstrip("/")}',
        f"--output_dir={eval_output_dir}",
        f"--apply_chat_template={apply_chat_template}",
        f"--batch_size={batch_size}",
        f"--huggingface_access_token={HF_TOKEN}",
    ]
    common_util.check_quota(
        project_id=PROJECT_ID,
        region=EVAL_REGION,
        accelerator_type=eval_accelerator_type,
        accelerator_count=eval_accelerator_count,
        is_for_training=True,
        is_dynamic_workload_scheduler=is_dynamic_workload_scheduler,
    )
    # Pass evaluation arguments and launch job.
    lm_eval_job = aiplatform.CustomContainerTrainingJob(
        display_name=common_util.get_job_name_with_datetime("llama3_1-lm-eval"),
        container_uri=LM_EVAL_DOCKER_URI,
        labels=labels,
    )

    print("Running evaluation job with args:")
    print(" \\\n".join(lm_eval_job_args))
    lm_eval_job.run(
        args=lm_eval_job_args,
        replica_count=1,
        machine_type=eval_machine_type,
        accelerator_type=eval_accelerator_type,
        accelerator_count=eval_accelerator_count,
        boot_disk_size_gb=boot_disk_size_gb,
        service_account=SERVICE_ACCOUNT,
        base_output_dir=base_output_dir,
        **dws_kwargs,
    )

# @markdown Click "Show Code" to see more details.

In [ ]:
# @title Deploy
# @markdown This section uploads the model to Model Registry and deploys it on the Endpoint. It takes 15 minutes to 1 hour to finish.

# The pre-built serving docker image for vLLM.
VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20250116_0916_RC00"

# @markdown Choose whether to use a [Spot VM](https://cloud.google.com/compute/docs/instances/spot) for the deployment.
is_spot = False  # @param {type:"boolean"}

# @markdown Set the Deployment Region. If not set, it will be set to default region.
DEPLOY_REGION = ""  # @param {type: "string"}
if not DEPLOY_REGION:
    DEPLOY_REGION = REGION

aiplatform.init(location=DEPLOY_REGION)

# Find Vertex AI prediction supported accelerators and regions [here](https://cloud.google.com/vertex-ai/docs/predictions/configure-compute).
if "8b" in base_model_id.lower():
    machine_type = "g2-standard-12"
    accelerator_type = "NVIDIA_L4"
    per_node_accelerator_count = 1
elif "70b" in base_model_id.lower():
    machine_type = "a3-highgpu-4g"
    accelerator_type = "NVIDIA_H100_80GB"
    per_node_accelerator_count = 4
elif "405b" in base_model_id.lower():
    machine_type = "a3-highgpu-8g"
    accelerator_type = "NVIDIA_H100_80GB"
    per_node_accelerator_count = 8
else:
    raise ValueError(f"Unsupported model ID or GCS path: {base_model_id}.")

common_util.check_quota(
    project_id=PROJECT_ID,
    region=DEPLOY_REGION,
    accelerator_type=accelerator_type,
    accelerator_count=per_node_accelerator_count,
    is_for_training=False,
)

# @markdown Set `use_dedicated_endpoint` to False if you don't want to use [dedicated endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment#create-dedicated-endpoint).
use_dedicated_endpoint = True  # @param {type:"boolean"}

gpu_memory_utilization = 0.95
max_model_len = 8192  # Maximum context length.

# Ensure max_model_len does not exceed the limit
if max_model_len > 8192:
    raise ValueError("max_model_len cannot exceed 8192")


def get_deploy_source() -> str:
    """Gets deploy_source string based on running environment."""
    vertex_product = os.environ.get("VERTEX_PRODUCT", "")
    if vertex_product == "COLAB_ENTERPRISE":
        return "notebook_colab_enterprise"
    elif vertex_product == "WORKBENCH_INSTANCE":
        return "notebook_workbench"
    else:
        # Legacy workbench, legacy colab, or other custom environments.
        return "notebook_environment_unspecified"


def deploy_model_vllm(
    model_name: str,
    model_id: str,
    publisher: str,
    publisher_model_id: str,
    service_account: str,
    base_model_id: str = None,
    machine_type: str = "g2-standard-8",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
    gpu_memory_utilization: float = 0.9,
    max_model_len: int = 4096,
    dtype: str = "auto",
    enable_trust_remote_code: bool = False,
    enforce_eager: bool = False,
    enable_lora: bool = False,
    enable_chunked_prefill: bool = False,
    enable_prefix_cache: bool = False,
    host_prefix_kv_cache_utilization_target: float = 0.0,
    max_loras: int = 1,
    max_cpu_loras: int = 8,
    use_dedicated_endpoint: bool = False,
    max_num_seqs: int = 256,
    model_type: str = None,
    enable_llama_tool_parser: bool = False,
    is_spot: bool = False,
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Deploys trained models with vLLM into Vertex AI."""
    endpoint = aiplatform.Endpoint.create(
        display_name=f"{model_name}-endpoint",
        dedicated_endpoint_enabled=use_dedicated_endpoint,
    )

    if not base_model_id:
        base_model_id = model_id

    # See https://docs.vllm.ai/en/latest/models/engine_args.html for a list of possible arguments with descriptions.
    vllm_args = [
        "python",
        "-m",
        "vllm.entrypoints.api_server",
        "--host=0.0.0.0",
        "--port=8080",
        f"--model={model_id}",
        f"--tensor-parallel-size={accelerator_count}",
        "--swap-space=16",
        f"--max-model-len={max_model_len}",
        f"--dtype={dtype}",
        f"--max-loras={max_loras}",
        f"--max-cpu-loras={max_cpu_loras}",
        f"--max-num-seqs={max_num_seqs}",
        "--disable-log-stats",
    ]

    if gpu_memory_utilization:
        vllm_args.append(f"--gpu-memory-utilization={gpu_memory_utilization}")

    if enable_trust_remote_code:
        vllm_args.append("--trust-remote-code")

    if enforce_eager:
        vllm_args.append("--enforce-eager")

    if enable_lora:
        vllm_args.append("--enable-lora")

    if enable_chunked_prefill:
        vllm_args.append("--enable-chunked-prefill")

    if enable_prefix_cache:
        vllm_args.append("--enable-prefix-caching")

    if 0 < host_prefix_kv_cache_utilization_target < 1:
        vllm_args.append(
            f"--host-prefix-kv-cache-utilization-target={host_prefix_kv_cache_utilization_target}"
        )

    if model_type:
        vllm_args.append(f"--model-type={model_type}")

    if enable_llama_tool_parser:
        vllm_args.append("--enable-auto-tool-choice")
        vllm_args.append("--tool-call-parser=vertex-llama-3")

    env_vars = {
        "MODEL_ID": base_model_id,
        "DEPLOY_SOURCE": "notebook",
    }

    # HF_TOKEN is not a compulsory field and may not be defined.
    try:
        if HF_TOKEN:
            env_vars["HF_TOKEN"] = HF_TOKEN
    except NameError:
        pass

    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=VLLM_DOCKER_URI,
        serving_container_args=vllm_args,
        serving_container_ports=[8080],
        serving_container_predict_route="/generate",
        serving_container_health_route="/ping",
        serving_container_environment_variables=env_vars,
        serving_container_shared_memory_size_mb=(16 * 1024),  # 16 GB
        serving_container_deployment_timeout=7200,
        model_garden_source_model_name=(
            f"publishers/{publisher}/models/{publisher_model_id}"
        ),
    )
    print(
        f"Deploying {model_name} on {machine_type} with {accelerator_count} {accelerator_type} GPU(s)."
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
        spot=is_spot,
        system_labels={
            "NOTEBOOK_NAME": "model_garden_pytorch_llama3_1_finetuning.ipynb",
            "NOTEBOOK_ENVIRONMENT": get_deploy_source(),
        },
    )
    print("endpoint_name:", endpoint.name)

    return model, endpoint


def predict_vllm(
    prompt: str,
    max_tokens: int,
    temperature: float,
    top_p: float,
    top_k: int,
    raw_response: bool,
    lora_weight: str = "",
):
    # Parameters for inference.
    instance = {
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "raw_response": raw_response,
    }
    if lora_weight:
        instance["dynamic-lora"] = lora_weight
    instances = [instance]
    response = endpoints["vllm_gpu"].predict(
        instances=instances, use_dedicated_endpoint=use_dedicated_endpoint
    )

    for prediction in response.predictions:
        print(prediction)


# Use FP8 base model for 405B since original model does not fit.
deploy_pretrained_model_id = pretrained_model_id
if "Meta-Llama-3.1-405B" in deploy_pretrained_model_id:
    deploy_pretrained_model_id += "-FP8"
print("Deploying model in:", deploy_pretrained_model_id)

models["vllm_gpu"], endpoints["vllm_gpu"] = deploy_model_vllm(
    model_name=common_util.get_job_name_with_datetime(prefix="llama3_1-vllm-serve"),
    model_id=deploy_pretrained_model_id,
    publisher="meta",
    publisher_model_id="llama3_1",
    service_account=SERVICE_ACCOUNT,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=per_node_accelerator_count,
    gpu_memory_utilization=gpu_memory_utilization,
    max_model_len=max_model_len,
    enable_lora=True,
    use_dedicated_endpoint=use_dedicated_endpoint,
    is_spot=is_spot,
)

# @markdown Click "Show Code" to see more details.

In [ ]:
# @title Predict

# @markdown Once deployment succeeds, you can send requests to the endpoint with text prompts. Sampling parameters supported by vLLM can be found [here](https://docs.vllm.ai/en/latest/dev/sampling_params.html).

# @markdown Example:

# @markdown ```
# @markdown Human: What is a car?
# @markdown Assistant:  A car, or a motor car, is a road-connected human-transportation system used to move people or goods from one place to another. The term also encompasses a wide range of vehicles, including motorboats, trains, and aircrafts. Cars typically have four wheels, a cabin for passengers, and an engine or motor. They have been around since the early 19th century and are now one of the most popular forms of transportation, used for daily commuting, shopping, and other purposes.
# @markdown ```

# @markdown Additionally, you can moderate the generated text with Vertex AI. See [Moderate text documentation](https://cloud.google.com/natural-language/docs/moderating-text) for more details.

prompt = "What is a car?"  # @param {type: "string"}
# @markdown If you encounter the issue like `ServiceUnavailable: 503 Took too long to respond when processing`, you can reduce the maximum number of output tokens, such as set `max_tokens` as 20.
max_tokens = 50  # @param {type:"integer"}
temperature = 1.0  # @param {type:"number"}
top_p = 1.0  # @param {type:"number"}
top_k = 1  # @param {type:"integer"}
raw_response = False  # @param {type:"boolean"}

predict_vllm(
    prompt=prompt,
    max_tokens=max_tokens,
    temperature=temperature,
    top_p=top_p,
    top_k=top_k,
    raw_response=raw_response,
    lora_weight=final_checkpoint,
)

# @markdown Click "Show Code" to see more details.

## Clean up resources

In [ ]:
# @title Delete the model and endpoint

if train_job:
    train_job.delete()
if RUN_EVALUATION and lm_eval_job:
    lm_eval_job.delete()


# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continuous charges that may incur.

# Undeploy model and delete endpoint.
for endpoint in endpoints.values():
    endpoint.delete(force=True)

# Delete models.
for model in models.values():
    model.delete()

delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_NAME